## Problem 5

In [ ]:
## For this question, fill in the TODO's 

In [1]:
# !pip install git+https://github.com/adapter-hub/adapter-transformers.git
!pip install adapter-transformers==1.0.1
!git clone https://github.com/huggingface/transformers
!python transformers/utils/download_glue_data.py --tasks RTE

import dataclasses
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Dict, Optional

import numpy as np

import torch
from transformers import AutoTokenizer, EvalPrediction, GlueDataset, GlueDataTrainingArguments, AutoModelWithHeads, AdapterType, AutoConfig, AutoModelForSequenceClassification
from transformers import GlueDataTrainingArguments as DataTrainingArguments
from transformers import (
    Trainer,
    TrainingArguments,
    glue_compute_metrics,
    glue_tasks_num_labels,
    set_seed,
)




     |████████████████████████████████| 686kB 9.0MB/s 
     |████████████████████████████████| 890kB 16.6MB/s 
     |████████████████████████████████| 1.1MB 31.6MB/s 
     |████████████████████████████████| 3.8MB 43.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=ff5ccfb5d049e84b09ecd0f68eb725448b5a663e7b7847fe262523604ac6749a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
Cloning into 'transformers'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 54622 (delta 43), reused 35 (delta 3), pack-reused 54542
Receiving objects: 100% (54622/54622), 40.78 MiB | 29.14 MiB/s, done.
Resolving deltas: 100% (38185/38185), done.
	Completed!


In [2]:
# model_name = "bert-base-uncased"
model_name = "roberta-base"

# Refer to the notebook for training an adapter to write these. Set the number of epochs to 3, and learning rate to 5e-5. Rest of the hyperparameters can stay the same. 

data_args = GlueDataTrainingArguments(task_name="rte", data_dir="./glue_data/RTE")
training_args = TrainingArguments(
    logging_steps=50, 
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=64, 
    save_steps=1000,
    evaluate_during_training=True,
    output_dir="./models/rte",
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    do_predict=True,
    learning_rate=5e-5,
    num_train_epochs=3,
)

# TODO: Change this seed when re-running your code to report the mean and std dev
set_seed(42)
num_labels = glue_tasks_num_labels[data_args.task_name]


In [3]:
config = AutoConfig.from_pretrained(
        model_name,
        num_labels=num_labels,
        finetuning_task=data_args.task_name,
        cache_dir=".",
    )

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir=".",
)

model = AutoModelWithHeads.from_pretrained(model_name, config=config)


Now we have everything set up to load our AdapterFusion setup. 

First, you need to go to adapterhub.ml and explore the different adapters available. Choose any three adapters and load the three adapters. As we don't need their prediction heads, we pass with_head=False to the loading method. Next, we add a new fusion layer that combines all the adapters we've just loaded. Finally, we add a new classification head for our target task on top.

In [4]:
# First, load the pre-trained adapters we want to fuse from Hub
from transformers.adapter_config import PfeifferConfig

model.load_adapter("nli/rte@ukp", "text_task", config=PfeifferConfig(), load_as='rte', with_head=False)
model.load_adapter("nli/multinli@ukp", 
                   AdapterType.text_task, 
                   load_as="multinli", 
                   with_head=False)
model.load_adapter("sts/qqp@ukp", 
                   AdapterType.text_task, 
                   with_head=False)
model.load_adapter("nli/qnli@ukp", 
                   AdapterType.text_task, 
                   with_head=False)
# TODO: load some more adapters. Choose the ones that you think will help RTE task after reading about the different tasks available and how big the datasets are.


# Add a fusion layer and tell the model to train fusion (freezes the rest of the weights) (here can either add the actual atsk adapter or not)
model.add_fusion([
        "rte","multinli", "qqp", "qnli"
        # TODO: Add your other task names here for the adapters you chose
    ])

# Add a classification head for our target task
model.add_classification_head("rte", num_labels=num_labels)
model.set_active_adapters([["rte", "multinli", "qqp", "qnli"]])
# TODO: Check the earlier notebook from Problem 5 to see how to add a classification head for your task.

The last preparation step is to define and activate our adapter setup. Similar to train_adapter(), train_fusion() does two things: It freezes all weights of the model (including adapters!) except for the fusion layer and classification head. It also activates the given adapter setup to be used in very forward pass.

The syntax for the adapter setup (which is also applied to other methods such as set_active_adapters()) works as follows:

a single string is interpreted as a single adapter, 
a list of strings is interpreted as a stack of adapters,
a nested list of strings is interpreted as a fusion of adapters. Here want to do Fusion so we use a nested list as follows. 


In [5]:
adapter_setup = [
                 [
        "rte", "multinli", "qqp", "qnli"
        # TODO: Add your other adapter names here. 
    ]
]
model.train_fusion(adapter_setup)

In [6]:
# Check out your training args
print(training_args)

TrainingArguments(output_dir='./models/rte', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=True, evaluate_during_training=True, per_device_train_batch_size=32, per_device_eval_batch_size=64, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, learning_rate=5e-05, weight_decay=0.0, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3, max_steps=-1, warmup_steps=0, logging_dir=None, logging_first_step=False, logging_steps=50, save_steps=1000, save_total_limit=None, no_cuda=False, seed=42, fp16=False, fp16_opt_level='O1', local_rank=-1, tpu_num_cores=None, tpu_metrics_debug=False)


In [8]:
train_dataset = GlueDataset(data_args, tokenizer=tokenizer)
eval_dataset = GlueDataset(data_args, tokenizer=tokenizer, mode="dev")

def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return glue_compute_metrics(data_args.task_name, preds, p.label_ids)

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
        do_save_full_model=False,
        do_save_adapter_fusion=True,
    )

# trainer.train()
# trainer.evaluate()

In [11]:
seed_list=[42, 77, 100]
for seed in seed_list:
    set_seed(seed)
    print("[SEED]", seed)
    config = AutoConfig.from_pretrained(
        model_name,
        num_labels=num_labels,
        finetuning_task=data_args.task_name,
        cache_dir=".",
    )

    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        cache_dir=".",
    )

    model = AutoModelWithHeads.from_pretrained(model_name, config=config)
    
    model.load_adapter("nli/rte@ukp", "text_task", config=PfeifferConfig(), load_as='rte', with_head=False)
    model.load_adapter("nli/multinli@ukp", 
                      AdapterType.text_task, 
                      load_as="multinli", 
                      with_head=False)
    model.load_adapter("sts/qqp@ukp", 
                      AdapterType.text_task, 
                      with_head=False)
    model.load_adapter("nli/qnli@ukp", 
                      AdapterType.text_task, 
                      with_head=False)
    model.train_fusion(adapter_setup)
    

    model.add_fusion([
            "rte","multinli", "qqp", "qnli"
            # TODO: Add your other task names here for the adapters you chose
        ])

    model.add_classification_head("rte", num_labels=num_labels)
    model.set_active_adapters([["rte", "multinli", "qqp", "qnli"]])

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
        do_save_full_model=False,
        do_save_adapter_fusion=True,
    )
    trainer.train()
    trainer.evaluate()

[SEED] 42


{"loss": 0.45350379079580305, "learning_rate": 3.931623931623932e-05, "epoch": 0.6410256410256411, "step": 50}



{"eval_loss": 0.6126170992851258, "eval_acc": 0.7436823104693141, "epoch": 0.6410256410256411, "step": 50}



{"loss": 0.33270268231630323, "learning_rate": 2.863247863247863e-05, "epoch": 1.282051282051282, "step": 100}



{"eval_loss": 0.9532286167144776, "eval_acc": 0.7220216606498195, "epoch": 1.282051282051282, "step": 100}
{"loss": 0.27083991147577763, "learning_rate": 1.794871794871795e-05, "epoch": 1.9230769230769231, "step": 150}


...


{"eval_loss": 0.7462080955505371, "eval_acc": 0.7328519855595668, "epoch": 1.282051282051282, "step": 100}
{"loss": 0.27735439300537107, "learning_rate": 1.794871794871795e-05, "epoch": 1.9230769230769231, "step": 150}



{"eval_loss": 0.6444199562072754, "eval_acc": 0.740072202166065, "epoch": 1.9230769230769231, "step": 150}



{"loss": 0.22870347917079925, "learning_rate": 7.264957264957266e-06, "epoch": 2.564102564102564, "step": 200}



{"eval_loss": 0.7867478251457214, "eval_acc": 0.7509025270758123, "epoch": 2.564102564102564, "step": 200}





{"eval_loss": 0.7171096682548523, "eval_acc": 0.7509025270758123, "epoch": 3.0, "step": 234}


In [15]:
train_loss = [0.2162778352200985, 0.21283287435770035, 0.22870347917079925]
val_loss = [0.7688331484794617, 0.7706224679946899, 0.7171096682548523]
val_acc = [0.7472924187725631, 0.7472924187725631, 0.7509025270758123]

print("train loss mean:{:.4f}, train loss std:{:.4f}".format(
     np.mean(train_loss), np.std(train_loss)))
print("val loss mean:{:.4f}, val loss std:{:.4f}".format(
    np.mean(val_loss), np.std(val_loss)
))
print("acc mean:{:.4f}, acc std:{:.4f}".format(
    np.mean(val_acc), np.std(val_acc)
))


train loss mean:0.2193, train loss std:0.0068
val loss mean:0.7522, val loss std:0.0248
acc mean:0.7485, acc std:0.0017
